In [30]:
import numpy as np
import cv2
import pytesseract
from glob import glob

In [2]:
def angle_cos(p0, p1, p2):
    d1, d2 = (p0-p1).astype('float'), (p2-p1).astype('float')
    return abs( np.dot(d1, d2) / np.sqrt( np.dot(d1, d1)*np.dot(d2, d2) ) )

def reorder_vertices(vertices):
    x_min = np.min(vertices[:,0])
    x_max = np.max(vertices[:,0])
    y_min = np.min(vertices[:,1])
    y_max = np.max(vertices[:,1])
    new_vertices = np.zeros((4,2), dtype=np.int32)
    new_vertices[0,0], new_vertices[0,1] = x_min, y_min
    new_vertices[1,0], new_vertices[1,1] = x_max, y_min
    new_vertices[2,0], new_vertices[2,1] = x_max, y_max
    new_vertices[3,0], new_vertices[3,1] = x_min, y_max
    return new_vertices

def find_buttons(img):
    img = cv2.GaussianBlur(img, (5, 5), 0)
    buttons = []
    for gray in cv2.split(img):
        for thrs in range(0, 255, 26):
            if thrs == 0:
                bin = cv2.Canny(gray, 0, 50, apertureSize=5)
                bin = cv2.dilate(bin, None)
            else:
                _retval, bin = cv2.threshold(gray, thrs, 255, cv2.THRESH_BINARY)
            contours, _hierarchy = cv2.findContours(bin, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
            for cnt in contours:
                cnt_len = cv2.arcLength(cnt, True)
                cnt = cv2.approxPolyDP(cnt, 0.02*cnt_len, True)
                if len(cnt) == 4 and cv2.contourArea(cnt) > 9000 and cv2.contourArea(cnt) < 40000 and cv2.isContourConvex(cnt):
#                     print('area: ', cv2.contourArea(cnt))
                    cnt = cnt.reshape(-1, 2)
                    max_cos = np.max([angle_cos( cnt[i], cnt[(i+1) % 4], cnt[(i+2) % 4] ) for i in range(4)])
                    if max_cos < 0.1:
                        cnt = reorder_vertices(cnt)
                        buttons.append(cnt)
    return buttons

def remove_duplicates(buttons):
    threshold = 50
    pos_y = [buttons[i][0][1] for i in range(len(buttons))]
    idx_y = np.argsort(pos_y)
    new_idx = []
    tmp_idx = []

    for i in range(len(idx_y)):
        if len(tmp_idx) == 0:
            tmp_idx.append(idx_y[i])
        if abs(pos_y[idx_y[i]] - pos_y[tmp_idx[-1]]) < threshold:
            if tmp_idx[-1] != idx_y[i]:
                tmp_idx.append(idx_y[i])
        else:
            pos_x = [buttons[j][0][0] for j in tmp_idx]
            idx_x = np.argsort(pos_x)
            tmp2_idx = []

            for j in range(len(idx_x)):
                if len(tmp2_idx) == 0:
                    tmp2_idx.append(tmp_idx[idx_x[j]])
                if abs(buttons[tmp_idx[idx_x[j]]][0][0] - buttons[tmp2_idx[-1]][0][0] ) > threshold:
                    tmp2_idx.append(tmp_idx[idx_x[j]])
#             print(len(tmp2_idx))
            new_idx.extend(tmp2_idx)
            tmp_idx = []
    pos_x = [buttons[j][0][0] for j in tmp_idx]
    idx_x = np.argsort(pos_x)
    tmp2_idx = []

    for j in range(len(idx_x)):
        if len(tmp2_idx) == 0:
            tmp2_idx.append(tmp_idx[idx_x[j]])
        if abs(buttons[tmp_idx[idx_x[j]]][0][0] - buttons[tmp2_idx[-1]][0][0] ) > threshold:
            tmp2_idx.append(tmp_idx[idx_x[j]])
#     print(len(tmp2_idx))
    new_idx.extend(tmp2_idx)
    new_buttons = [buttons[i] for i in new_idx]
    return new_buttons

In [29]:
fn = 'data/pic_comm5_home.png'
# fn = 'data/pic_comm5_things.png'
img = cv2.imread(fn)
buttons = find_buttons(img)
buttons = remove_duplicates(buttons)
print('num buttons: ', len(buttons))
cv2.drawContours( img, buttons, -1, (0, 255, 0), 3 )
cv2.imshow('buttons', img)
ch = cv2.waitKey()

print('Done')
cv2.destroyAllWindows()

num buttons:  33
Done


In [ ]:
# for i in range(len(squares)):
#     print('area: ', cv.contourArea(squares[i]))

In [28]:
for i in range(len(buttons)):
    cnt = buttons[i]
    x,y,w,h = cv2.boundingRect(cnt)
    crop = img[y:y+h,x:x+w]

    # Check if button is empty --> 96% pixels are white
    num_white = np.sum((crop[:,:,0] >= 200) & (crop[:,:,1] >= 200) & (crop[:,:,2] >= 200))
    num_pixels = np.prod(crop.shape[:2])
    # print(num_white)
    # print(num_pixels)
    print('%d:\t%0.2f' % (i, num_white/num_pixels*100))
    
    # Find text
#     text = pyter

    # cv2.imshow('cropped', crop)
    # ch = cv2.waitKey()
    # cv2.destroyAllWindows()

0:	3.97
1:	3.77
2:	91.78
3:	11.08
4:	11.65
5:	27.68
6:	0.02
7:	3.88
8:	0.00
9:	0.00
10:	0.00
11:	1.00
12:	0.00
13:	0.00
14:	0.00
15:	4.33
16:	0.30
17:	0.03
18:	72.58
19:	73.98
20:	69.20
21:	73.27
22:	69.42
23:	74.87
24:	68.17
25:	73.97
26:	67.27
27:	74.54
28:	74.08
29:	68.64
30:	69.66
31:	74.38
32:	76.40


8655
11684
0.7407565902088326
